# Q1: Simple CNN network for PASCAL multi-label classification (20 points)
Now let’s try to recognize some natural images. We provided some starter code for this task. The following steps will guide you through the process.


## 1.1 Setup the dataset
We start by modifying the code to read images from the PASCAL 2007 dataset. The important thing to note is that PASCAL can have multiple objects present in the same image. Hence, this is a multi-label classification problem, and will have to be tackled slightly differently.


First, download the data. `cd` to a location where you can store 0.5GB of images. Then run:
```
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
tar -xf VOCtrainval_06-Nov-2007.tar

wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
tar -xf VOCtest_06-Nov-2007.tar
cd VOCdevkit/VOC2007/
```

## 1.2 Write a dataloader with data augmentation (5 pts)
**Dataloader** The first step is to write a [pytorch data loader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) which loads this PASCAL data. Complete the functions `preload_anno` and `__getitem__` in `voc_dataset.py`. 
* **Hint**: Refer to the `README` in `VOCdevkit` to understand the structure and labeling.
* **Hint** : As the function docstring says, `__getitem__` takes as input the index, and returns a tuple - `(image, label, weight)`. The labels should be 1s for each object that is present in the image, and weights should be 1 for each label in the image, except those labeled as ambiguous (use the `difficult` attribute). All other values should be 0. For simplicity, resize all images to a canonical size.)


**Data Augmentation** Modify `__getitem__` to randomly *augment* each datapoint. Please describe what data augmentation you implement.
* **Hint**: Since we are training a model from scratch on this small dataset, it is important to perform basic data augmentation to avoid overfitting. Add random crops and left-right flips when training, and do a center crop when testing, etc. As for natural images, another common practice is to subtract the mean values of RGB images from ImageNet dataset. The mean values for RGB images are: `[123.68, 116.78, 103.94]` – sometimes, rescaling to `[−1, 1]` suffices.

**Note:** You should use data in ‘trainval’ for training and ‘test’ for testing, since PASCAL is a small dataset.


### DESCRIBE YOUR AUGMENTATION PIPELINE HERE**
**Train Augmentations:**

The following augmentations were performed on the training set:

* Rescaling to 28 x 28
* Random horizontal flipping
* Normalization based on a mean of (0.485, 0.456, 0.406) and a standard deviation of (0.229, 0.224, 0.225)

<br>

**Test Augmentations:**

The following augmentations were performed on the test set:

* Rescaling to 28 x 28
* Random horizontal flipping
* Normalization based on a mean of (0.485, 0.456, 0.406) and a standard deviation of (0.229, 0.224, 0.225)


## 1.3 Measure Performance (5 pts)
To evaluate the trained model, we will use a standard metric for multi-label evaluation - [mean average precision (mAP)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). Please implement `eval_dataset_map` in `utils.py` - this function will evaluate a model's map score using a given dataset object. You will need to make predictions on the given dataset with the model and call `compute_ap` to get average precision.


Please describe how to compute AP for each class(not mAP).
**YOUR ANSWER HERE**

The average precision (AP) is computed by gettings the ground truth, non-difficult classes and the correct predition classes. The predicition class get a small decimal subtracted from it (perhaps for normalization). Then an average precision score is calculated between the two vectors and given a value for each class and stacked on top of each of as a vector. At the end of the function, the AP vector is returned out.


## 1.4 Let's Start Training! (5 pts)
Write the code for training and testing for multi-label classification in `trainer.py`. To start, you'll use the same model you used for Fashion MNIST (bad idea, but let’s give it a shot).

Initialize a fresh model and optimizer. Then run your training code for 5 epochs and print the mAP on test set.

In [1]:
import torch
import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset

# create hyperparameter argument class
args = ARGS(epochs=5)
print(args)

args.batch_size = 32
args.device = cuda
args.epochs = 5
args.gamma = 0.7
args.log_every = 100
args.lr = 1.0
args.save_at_end = False
args.save_freq = 10
args.test_batch_size = 1000
args.val_every = 100



In [ ]:
# initializes (your) naiive model
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
# initializes Adam optimizer and simple StepLR scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.gamma)
# trains model using your training code and reports test map
test_ap, test_map = trainer.train(args, model, optimizer, scheduler)
print('test map:', test_map)

[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is an awesome visualization tool. It was firstly integrated in [TensorFlow](https://www.tensorflow.org/) (~possibly the only useful tool TensorFlow provides~). It can be used to visualize training losses, network weights and other parameters.

To use TensorBoard in Pytorch, there are two options: [TensorBoard in Pytorch](https://pytorch.org/docs/stable/tensorboard.html) (for Pytorch >= 1.1.0) or [TensorBoardX](https://github.com/lanpa/tensorboardX) - a third party library. Add code in `trainer.py` to visualize the testing MAP and training loss in Tensorboard. *You may have to reload the kernel for these changes to take effect*

Show clear screenshots of the learning curves of testing MAP and training loss for 5 epochs (batch size=20, learning rate=0.001). Please evaluate your model to calculate the MAP on the testing dataset every 100 iterations.

In [2]:
args = ARGS(epochs=5, batch_size=20, lr=0.001)
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args.gamma)
if __name__ == "__main__":
    test_ap, test_map = trainer.train(args, model, optimizer, scheduler)
    print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.694451 | mAP: 0.076841
Train Epoch: 0 [100 (40%)]	Loss: 0.240711 | mAP: 0.123666
Train Epoch: 0 [200 (80%)]	Loss: 0.242937 | mAP: 0.163159
Train Epoch: 1 [300 (20%)]	Loss: 0.212498 | mAP: 0.173409
Train Epoch: 1 [400 (59%)]	Loss: 0.211122 | mAP: 0.193135
Train Epoch: 1 [500 (99%)]	Loss: 0.216955 | mAP: 0.204963
Train Epoch: 2 [600 (39%)]	Loss: 0.186005 | mAP: 0.218065
Train Epoch: 2 [700 (79%)]	Loss: 0.140446 | mAP: 0.224668
Train Epoch: 3 [800 (19%)]	Loss: 0.208131 | mAP: 0.232051
Train Epoch: 3 [900 (59%)]	Loss: 0.194046 | mAP: 0.234823
Train Epoch: 3 [1000 (98%)]	Loss: 0.199973 | mAP: 0.238088
Train Epoch: 4 [1100 (38%)]	Loss: 0.199175 | mAP: 0.241002
Train Epoch: 4 [1200 (78%)]	Loss: 0.167286 | mAP: 0.246116
test map: 0.24888071523104266


**INSERT YOUR TENSORBOARD SCREENSHOTS HERE**

The figures below display the learning rate, training loss, and training map, respectively:

![title](img/q1_img01_learningrate.JPG)

![title](img/q1_img02_loss.JPG)

![title](img/q1_img03_map.JPG)